In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install unsloth bitsandbytes datasets accelerate trl -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from huggingface_hub import login
login(token="")  # 🔑 Replace with your actual Hugging Face token


In [35]:
!pip install wandb -q


In [38]:
import os
os.environ["WANDB_API_KEY"] = ""


In [4]:
import torch
from unsloth import FastVisionModel
from transformers import TextStreamer
from datasets import load_dataset
from unsloth.trainer import UnslothVisionDataCollator
from unsloth import is_bf16_supported
from trl import SFTTrainer, SFTConfig

# Load pre-trained vision model
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit=True,
    use_gradient_checkpointing="unsloth",
)

# Apply LoRA adapters
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers=True,
    finetune_language_layers=True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Mllama patching. Transformers: 4.50.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/375k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

Unsloth: Making `model.base_model.model.vision_model.transformer` require gradients


In [8]:
from PIL import Image

image_path = "/content/drive/MyDrive/200set/200set_exclude_test_image/row_101_teacher.png"
image = Image.open(image_path).convert("RGB")


In [10]:
FastVisionModel.for_inference(model)

instruction = (
     "You are a vision-language assistant. Describe an image using exactly six categories in a single line:\n\n"
        "main_objects: ... main_object_attributes: ... location: ... action: ... surroundings: ... background: ...\n\n"
        "Format rules:\n"
        "- Each category must start with its name, followed by a colon and a space\n"
        "- Use detailed, specific descriptions\n"
        "- Separate categories with a comma and a space\n"
        "- If a category is unclear, write: [category name]: none\n"
        "- Do NOT include commentary, line breaks, or extra text\n\n"
        "Example:\n"
        "main_objects: rabbit main_object_attributes: brown fur, fluffy tail, big ears, happy expression location: meadow action: none surroundings: butterflies, flowers, green grass background: foliage, sunlight"
)

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

inputs = tokenizer(
    image,
    input_text,
    return_tensors="pt",
    add_special_tokens=False,
).to("cuda")

streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(**inputs, streamer=streamer, max_new_tokens=128, temperature=1.0)


main_objects: Woman, book main_object_attributes: glasses, long brown hair, red lipstick, white shirt, blue jacket, reading action: reading location: library surroundings: bookshelves, candles background: shelves, books, dimly lit room<|eot_id|>


In [21]:
import os
import pandas as pd
from shutil import copy2

# === CONFIG ===
image_dir = "/content/drive/MyDrive/200set/200set_exclude_test_image"
csv_path = "/content/drive/MyDrive/200set/left 200.csv"
output_image_dir = "/content/drive/MyDrive/200set/renamed_images"
output_csv_path = "/content/drive/MyDrive/200set/renamed_captions.csv"
os.makedirs(output_image_dir, exist_ok=True)

# Load CSV
df = pd.read_csv(csv_path, delimiter="\t" if "\t" in open(csv_path).readline() else ",")

# Output list
output_rows = []
counter = 1

for _, row in df.iterrows():
    row_id = str(row["Row"])

    for role in ["Teacher", "Student"]:
        old_filename = f"row_{row_id.lower()}_{role.lower()}.png"
        new_filename = f"{counter:04d}.png"

        src_path = os.path.join(image_dir, old_filename)
        dst_path = os.path.join(output_image_dir, new_filename)

        if os.path.exists(src_path):
            copy2(src_path, dst_path)
            output_rows.append({"filename": new_filename, "caption": row[role]})
            counter += 1
        else:
            print(f"⚠️ File not found: {src_path}")

# Save new captions CSV
pd.DataFrame(output_rows).to_csv(output_csv_path, index=False)

print("✅ Renaming complete.")
print(f"📁 Images saved to: {output_image_dir}")
print(f"📝 Captions saved to: {output_csv_path}")


✅ Renaming complete.
📁 Images saved to: /content/drive/MyDrive/200set/renamed_images
📝 Captions saved to: /content/drive/MyDrive/200set/renamed_captions.csv


In [29]:
import pandas as pd
import re

# === CONFIG ===
csv_path = "/content/drive/MyDrive/200set/renamed_captions.csv"

# === LOAD CSV ===
df = pd.read_csv(csv_path)

# === FUNCTION TO FIX CAPTIONS ===
def normalize_caption_format(caption: str) -> str:
    caption = caption.strip()
    replacements = {
        r"\bMain Object[s]?:": "main_objects:",
        r"\bAttributes:": "main_object_attributes:",
        r"\blocation:": "location:",
        r"\baction:": "action:",
        r"\bsurroundings:": "surroundings:",
        r"\bbackground:": "background:",
    }
    for pattern, replacement in replacements.items():
        caption = re.sub(pattern, replacement, caption, flags=re.IGNORECASE)
    return caption

# === APPLY FIX TO ALL CAPTIONS ===
df["caption"] = df["caption"].astype(str).apply(normalize_caption_format)

# === SAVE BACK TO SAME FILE ===
df.to_csv(csv_path, index=False)
print("✅ CSV captions normalized and saved to the same file.")


✅ CSV captions normalized and saved to the same file.


In [4]:
BASE_PROMPT = (
    "You are a vision-language assistant. Describe an image using exactly six categories in a single line:\n\n"
    "main_objects: ... main_object_attributes: ... location: ... action: ... surroundings: ... background: ...\n\n"
    "Format rules:\n"
    "- Each category must start with its name, followed by a colon and a space\n"
    "- Use detailed, specific descriptions\n"
    "- Separate categories with a comma and a space\n"
    "- If a category is unclear, write: [category name]: none\n"
    "- Do NOT include commentary, line breaks, or extra text\n\n"
    "Example:\n"
    "main_objects: rabbit main_object_attributes: brown fur, fluffy tail, big ears, happy expression location: meadow action: none surroundings: butterflies, flowers, green grass background: foliage, sunlight"
)


In [6]:
import pandas as pd
from datasets import Dataset
import os

# === CONFIG ===
csv_path = "/content/drive/MyDrive/200set/renamed_captions.csv"
image_dir = "/content/drive/MyDrive/200set/renamed_images"
BASE_PROMPT = (
    "You are a vision-language assistant. Describe an image using exactly six categories in a single line:\n\n"
    "main_objects: ... main_object_attributes: ... location: ... action: ... surroundings: ... background: ...\n\n"
    "Format rules:\n"
    "- Each category must start with its name, followed by a colon and a space\n"
    "- Use detailed, specific descriptions\n"
    "- Separate categories with a comma and a space\n"
    "- If a category is unclear, write: [category name]: none\n"
    "- Do NOT include commentary, line breaks, or extra text\n\n"
    "Example:\n"
    "main_objects: rabbit main_object_attributes: brown fur, fluffy tail, big ears, happy expression location: meadow action: none surroundings: butterflies, flowers, green grass background: foliage, sunlight"
)

# === LOAD CSV ===
df = pd.read_csv(csv_path)

# === CONVERT TO CONVERSATION FORMAT (with "file://" paths) ===
samples = []
for idx, row in df.iterrows():
    img_path = os.path.join(image_dir, row["filename"])

    if not os.path.exists(img_path):
        print(f"⚠️ Skipping missing image: {img_path}")
        continue
    if not isinstance(row["caption"], str) or not row["caption"].strip():
        print(f"⚠️ Skipping empty caption at index {idx}")
        continue

    sample = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": BASE_PROMPT},
                    {"type": "image", "image": f"file://{img_path}"}  # ✅ The key fix
                ]
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": row["caption"].strip()}
                ]
            }
        ]
    }
    samples.append(sample)

# === CONVERT TO HF DATASET ===
dataset = Dataset.from_list(samples)
print(f"✅ Final dataset created with {len(dataset)} samples.")

# === ✅ Upload to Hugging Face Hub ===
dataset.push_to_hub("wolf010/vision_caption_200set", private=False)


✅ Final dataset created with 400 samples.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/wolf010/vision_caption_200set/commit/7dcad5849884b2469a46f9e412247d77b896f6c3', commit_message='Upload dataset', commit_description='', oid='7dcad5849884b2469a46f9e412247d77b896f6c3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/wolf010/vision_caption_200set', endpoint='https://huggingface.co', repo_type='dataset', repo_id='wolf010/vision_caption_200set'), pr_revision=None, pr_num=None)

In [7]:
from datasets import Dataset
import pandas as pd
import os

csv_path = "/content/drive/MyDrive/200set/renamed_captions.csv"
image_dir = "/content/drive/MyDrive/200set/renamed_images"

df = pd.read_csv(csv_path)
samples = []

for _, row in df.iterrows():
    img_path = os.path.join(image_dir, row["filename"])

    if not os.path.exists(img_path):
        continue

    with open(img_path, "rb") as f:
        img_bytes = f.read()

    sample = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": BASE_PROMPT},
                    {"type": "image", "image": {"bytes": img_bytes}}
                ]
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": row["caption"].strip()}
                ]
            }
        ]
    }
    samples.append(sample)

dataset = Dataset.from_list(samples)

# === ✅ Upload to Hugging Face Hub ===
dataset.push_to_hub("wolf010/vision_caption_200set", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/458 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/wolf010/vision_caption_200set/commit/c21a1f437fc91fb662da2852ba7639c60f128c25', commit_message='Upload dataset', commit_description='', oid='c21a1f437fc91fb662da2852ba7639c60f128c25', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/wolf010/vision_caption_200set', endpoint='https://huggingface.co', repo_type='dataset', repo_id='wolf010/vision_caption_200set'), pr_revision=None, pr_num=None)

In [66]:
from PIL import Image
from IPython.display import display
import io

for i in range(3):
    print(f"\n=== SAMPLE {i+1} ===")
    print("🗨️ Prompt:\n", dataset[i]["messages"][0]["content"][0]["text"])

    # Retrieve the image content
    img_content = dataset[i]["messages"][0]["content"][1]["image"]

    # If it's a path (string), open normally
    if isinstance(img_content, str):
        img = Image.open(img_content)

    # If it's a dictionary with 'bytes', load from memory
    elif isinstance(img_content, dict) and "bytes" in img_content:
        img = Image.open(io.BytesIO(img_content["bytes"]))

    # Display the image
    display(img)

    print("🧠 Caption:\n", dataset[i]["messages"][1]["content"][0]["text"])



=== SAMPLE 1 ===
🗨️ Prompt:
 You are a vision-language assistant. Describe an image using exactly six categories in a single line:

main_objects: ... main_object_attributes: ... location: ... action: ... surroundings: ... background: ...

Format rules:
- Each category must start with its name, followed by a colon and a space
- Use detailed, specific descriptions
- Separate categories with a comma and a space
- If a category is unclear, write: [category name]: none
- Do NOT include commentary, line breaks, or extra text

Example:
main_objects: rabbit main_object_attributes: brown fur, fluffy tail, big ears, happy expression location: meadow action: none surroundings: butterflies, flowers, green grass background: foliage, sunlight


FileNotFoundError: [Errno 2] No such file or directory: 'file:///content/drive/MyDrive/200set/renamed_images/0001.png'

In [44]:
FastVisionModel.for_training(model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MllamaForConditionalGeneration(
      (vision_model): MllamaVisionModel(
        (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), padding=valid, bias=False)
        (gated_positional_embedding): MllamaPrecomputedPositionEmbedding(
          (tile_embedding): Embedding(9, 8197120)
        )
        (pre_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
          (embedding): Embedding(9, 5120)
        )
        (post_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
          (embedding): Embedding(9, 5120)
        )
        (layernorm_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (layernorm_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (transformer): MllamaVisionEncoder(
          (layers): ModuleList(
            (0-12): 13 x MllamaVisionEncoderLayer(
              (self_attn): MllamaVisionSdpaAttention(
               

In [67]:
from trl import SFTTrainer, SFTConfig
from unsloth.trainer import UnslothVisionDataCollator

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    data_collator=UnslothVisionDataCollator(model, tokenizer),
    args=SFTConfig(
        output_dir="/content/drive/MyDrive/llama_vision_finetuned",  # ✅ Save final model
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        max_steps=250,
        learning_rate=2e-4,
        warmup_steps=5,
        fp16=not is_bf16_supported(),
        bf16=is_bf16_supported(),
        logging_steps=1,  # 📊 Log every step (great for small datasets)
        save_strategy="epoch",  # 💾 Optionally change this to "no" if you don't want intermediate saves
        evaluation_strategy="no",  # 👈 Add this if you don't have a `eval_dataset`
        report_to="wandb",  # ✅ Enable wandb logging
        run_name="llama_vision_finetune",  # 📛 Optional, nice for identifying runs
        max_seq_length=2048,
        seed=42,
        dataset_text_field="",
        dataset_num_proc=4,
        remove_unused_columns=False,
        dataset_kwargs={"skip_prepare_dataset": True},
    )
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [68]:
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 400 | Num Epochs = 5 | Total steps = 250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 67,174,400/11,000,000,000 (0.61% trained)


AttributeError: 'NoneType' object has no attribute 'startswith'